# Barbershop Model - Binary programming

Joaquín Rodríguez Villegas

## Sets 

* $I$: Set of Barbershops, i $\in$ {1,2,…,I} 

## Parameters

* $d_{i}$: Is the distance between home and the barbershop ${i}$, $\forall i \in I$
* $c_{i}$: Is the general service haircut cost (price) at the barbershop ${i}$, $\forall i \in I$
* $TE_{i}$: Is the average waiting time at the barbershop ${i}$, $\forall i \in I$
* $TS_{i}$: Is the average service time at the barbershop ${i}$, $\forall i \in I$
* $Budget$: Is the available budget

## Decision variables

$$ x_{i}: 
    \begin{cases}
      1 \space\text{if the barbershop ${i}$ is chosen,} \space \forall i \in I \\
      0 \space\text{if the barbershop ${i}$ is not chosen} \space \forall i \in I
    \end{cases} $$

## Objective functions

Cost objective:
$$ \ Min\space \ Z = \sum_{i \in I} c_{i}*x_{i} $$

Distance objective:
$$ \ Min\space \ Z = \sum_{i \in I} d_{i}*x_{i} $$

Time objective:
$$ \ Min\space \ Z = \sum_{i \in I} \frac{1}{2}(TE_{i} +TS_{i})*x_{i} $$

## Constraints

#### Only one barbershop constraint:
$$ \sum_{i \in I} x_{i} = 1$$

#### Budget constraint:
$$ \sum_{i \in I} c_{i}*x_{i} \leq Budget$$

#### Nonnegative constraints
$$ x_{i} \in \{0,1\},\forall i \in I $$



In [1]:
#Import libraries
import pyomo.environ as pe
import pyomo.opt as po

In [2]:
#Build data
barbershops = {'Norberto','Blanquita','Vidal','Beerberia'}
distance = {'Norberto':0.6125,'Blanquita':6.12,'Vidal':0.42709,'Beerberia': 0.54045}
cost = {'Norberto':36000,'Blanquita':15000,'Vidal':30000,'Beerberia':38000}
Budget = 40000

In [3]:
#Optimization Model
#Sets
model = pe.ConcreteModel()
model.barbershop = pe.Set(initialize = barbershops)

#Parameters
model.distance = pe.Param(model.barbershop, initialize = distance)
model.cost = pe.Param(model.barbershop, initialize = cost)
model.Budget = pe.Param(initialize = Budget)

#Decision variables
model.x = pe.Var(model.barbershop, domain = pe.Binary)

#Objective functions
def calculate_cost(model):
    '''
    This function calculates the total cost at the barbershop.

    Parameters
    ----------
    model : Pyomo ConcreteModel
        The optimization model.

    Return
    ------------
    double
        Barbershop costs
    '''
    cost = sum(model.cost[i]* model.x[i] for i in model.barbershop)
    return cost

def calculate_distance(model):
    '''
    This function calculates the total distance to the barbershop.

    Parameters
    ----------
    model : Pyomo ConcreteModel
        The optimization model.

    Return
    ------------
    double
        Barbershop distance
    '''
    distance = sum(model.distance[i]* model.x[i] for i in model.barbershop)
    return distance

#Constraints
def one_barbershop_constraint(model):
    '''
    Constraint that defines if a given barbershop is chosen or not, there can only be one
    barbershop to be chosen from.

    Parameters
    ----------
    model : Pyomo ConcreteModel
        The optimization model.

    Returns
    -------
    Pyomo ConcreteModel
        The optimization model.
    '''
    left_hand_side  = sum([model.x[i] for i in model.barbershop]) 
    right_hand_side = 1

    return left_hand_side == right_hand_side


def budget_constraint(model):
    '''
    Constraint that defines if a given barbershop is chosen or not, there can only be one
    barbershop to be chosen from.

    Parameters
    ----------
    model : Pyomo ConcreteModel
        The optimization model.

    Returns
    -------
    Pyomo ConcreteModel
        The optimization model.
    '''
    left_hand_side  = sum(model.cost[i]* model.x[i] for i in model.barbershop)
    right_hand_side = model.Budget

    return left_hand_side <= right_hand_side

    data source (type: set).  This WILL potentially lead to nondeterministic
    behavior in Pyomo


In [4]:
#model.obj_function_cost = pe.Objective(sense = pe.minimize, rule = calculate_cost)
model.obj_function_distance = pe.Objective(sense = pe.minimize, rule = calculate_distance)
model.add_one_barbershop_constraint = pe.Constraint(rule=one_barbershop_constraint)
model.add_budget_constraint = pe.Constraint(rule=budget_constraint)

solver = pe.SolverFactory('glpk')
result = solver.solve(model, tee=True) 
#pe.value(model.obj_function_cost)
pe.value(model.obj_function_distance)

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write C:\Users\JRODRI~1.VIL\AppData\Local\Temp\tmpipeu69bp.glpk.raw --wglp
 C:\Users\JRODRI~1.VIL\AppData\Local\Temp\tmp22no19fi.glpk.glp --cpxlp C:\Users\JRODRI~1.VIL\AppData\Local\Temp\tmpz6rkcut2.pyomo.lp
Reading problem data from 'C:\Users\JRODRI~1.VIL\AppData\Local\Temp\tmpz6rkcut2.pyomo.lp'...
C:\Users\JRODRI~1.VIL\AppData\Local\Temp\tmpz6rkcut2.pyomo.lp:35: warning: lower bound of variable 'x1' redefined
C:\Users\JRODRI~1.VIL\AppData\Local\Temp\tmpz6rkcut2.pyomo.lp:35: warning: upper bound of variable 'x1' redefined
3 rows, 5 columns, 9 non-zeros
4 integer variables, all of which are binary
39 lines were read
Writing problem data to 'C:\Users\JRODRI~1.VIL\AppData\Local\Temp\tmp22no19fi.glpk.glp'...
28 lines were written
GLPK Integer Optimizer 5.0
3 rows, 5 columns, 9 non-zeros
4 integer variables, all of which are binary
Preprocessing...
1 hidden packing inequaliti(es) were detected
2 rows, 4 columns, 

0.42709

In [5]:
model.x.pprint()

x : Size=4, Index=barbershop
    Key       : Lower : Value : Upper : Fixed : Stale : Domain
    Beerberia :     0 :   0.0 :     1 : False : False : Binary
    Blanquita :     0 :   0.0 :     1 : False : False : Binary
     Norberto :     0 :   0.0 :     1 : False : False : Binary
        Vidal :     0 :   1.0 :     1 : False : False : Binary
